In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import numpy
import pandas
from IPython.display import display
import custom
from kiwipiepy import Kiwi
from tqdm import tqdm


In [2]:
kiwi = Kiwi()

print(kiwi.tokenize("안녕하세요"))
print(kiwi.tokenize("진짜 조낸 재밌다 굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿"))

[Token(form='안녕', tag='NNG', start=0, len=2), Token(form='하', tag='XSA', start=2, len=1), Token(form='세요', tag='EF', start=3, len=2)]
[Token(form='진짜', tag='MAG', start=0, len=2), Token(form='조', tag='NNP', start=3, len=1), Token(form='내', tag='NNB', start=4, len=1), Token(form='ᆫ', tag='JX', start=4, len=1), Token(form='재밌', tag='VA', start=6, len=2), Token(form='다', tag='EF', start=8, len=1), Token(form='굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿', tag='NNG', start=10, len=22)]


In [3]:
original_df = pandas.read_csv("data/ratings_train.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [4]:
df = original_df[['document','label']]

for i in tqdm(range(len(df))) :
    text = custom.text_preprocess_kor(str(df.iloc[i,0]), end_mark = True)
    if text == " " :
        df.iloc[i,0] = text
        continue
    try :
        temp = kiwi.tokenize(text)
        temp_list = []
        for w in temp :
            temp_list.append(w.form)
        df.iloc[i,0] = " ".join(temp_list)
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}")
    
word_len = []
for i in range(len(df)) :
    word_len.append(len(str(df.iloc[i,0]).split()))
df["word_len"] = word_len

df = df.drop(df[df["word_len"] == 0].index, axis = 0)

display(df)


100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [05:57<00:00, 419.95it/s]


,document,label,word_len
0,아 더빙 . 진짜 짜증 나 네요 목소리,0,8
1,흠 . 포스터 보고 초딩 영화 줄 . 오버 연기 조차 가볍 지 않 구나,1,15
2,너무재밓 었 다 그래서 보 는 것 을 추천 하 ᆫ다,0,11
3,교도소 이야기 이 구먼 . 솔직히 재미 는 없 다 . 평점 조정,0,13
4,사이몬페그 의 익살 스럽 ᆫ 연기 가 돋보이 었 던 영화 ! 스파이더맨 에서 늙 어...,1,32
...,...,...,...
149995,인간 이 문제 이 지 . 소 는 뭔 죄 이 ᆫ가 .,0,13
149996,평점 이 너무 낮 어서 .,1,6
149997,이것 이 뭐 요 ? 한국인 은 거들먹거리 고 필리핀 혼혈 은 착하 다 ?,0,15
149998,청춘 영화 의 최 고봉 . 방황 과 우울 하 었 던 날 들 의 자화상,1,16


In [5]:

df.to_csv("data/korean_movie_train.txt", sep = "\t", encoding = "UTF8")

In [6]:
print(df["word_len"].max())
print(df["word_len"].mean())
print(df["word_len"].std())

112
19.021193182199607
16.3219540727404
